In [ ]:
import pandas as pd
import numpy as np
import itertools
from scipy.stats import multivariate_normal as mn
from scipy.stats import norm
import statsmodels.api as sm
import warnings

In [ ]:
def srivastava(data):
    '''
    Inputs:
        data: np.ndarray
            Residual correlation matrix
    Performs:
        Perform test T3* from Srivastava (2005) to test wheter
        the corrleation matrix is diagonal
    Returns:
        float
    '''
    n = data.shape[0]
    p = data.shape[1]
    if p > 1:
        S = np.cov(data.T)
        a2_hat = np.sum(np.square(np.diag(S)))
        a4_hat = np.sum(np.power(np.diag(S), 4))
        a20_hat = (n/(p*(n+2)))*a2_hat
        a40_hat = (1/p)*a4_hat
        g3_hat = (n/(n-1))*((np.trace(np.dot(S,S))-(1/n)*(np.trace(S))**2)/(np.sum(np.square(np.diag(S)))))

        with warnings.catch_warnings():
            warnings.filterwarnings('ignore')
            T3_hat = (n/2)*((g3_hat-1)/np.sqrt(1-(1/p)*(a40_hat/(a20_hat**2))))
            if np.isnan(T3_hat): # Could have sqrt of negative, replace as in Wang et al.
                T3_hat = (n/2)*((g3_hat-1)/np.sqrt(1-(a4_hat/(a2_hat**2))))

        return T3_hat
    else: # Test isn't meaningful, so do not exclude the model on this basis
        return 0 

In [ ]:
cors = [0]
ms = [5, 25, 50]
ns = [10, 100, 10000]
ks = [1000]
alphas = [0.01, 0.05]

results = pd.DataFrame()

for cor, m, n, k, alpha in itertools.product(cors, ms, ns, ks, alphas):
    rejected = 0
    for i in range(k):
        cov = np.identity(m)
        cov[cov == 0] = cor
        sample = mn.rvs(cov=cov, size=n).reshape(n, -1)

        t = srivastava(sample)
        # if not valid:
        if np.abs(t) > norm.ppf(1-(alpha/2)):
            rejected += 1

    emp_alpha = rejected/k

    result = {'c': cor, 'm': m, 'n': n, 'k': k, 'a': alpha, 'es': emp_alpha, 'd': np.abs(alpha)-emp_alpha}
    results = results.append(result, ignore_index=True)

results = results.loc[:,['es', 'a', 'd', 'n', 'c', 'm', 'k']]

In [ ]:
results['n'] = results['n'].astype(int)
results['m'] = results['m'].astype(int)
results['k'] = results['k'].astype(int)

In [ ]:
results.to_csv('../text/latex/algo/files/test_validation_alpha.csv', index=False, float_format='%.3f')

In [ ]:
cors = np.linspace(0.1, 1, 5)
ms = [5, 25, 50]
ns = [10, 100, 10000]
ks = [1000]
alphas = [0.01, 0.05]

results = pd.DataFrame()

for cor, m, n, k, alpha in itertools.product(cors, ms, ns, ks, alphas):
    rejected = 0
    for i in range(k):
        cov = np.identity(m)
        cov[cov == 0] = cor
        sample = mn.rvs(cov=cov, size=n).reshape(n, -1)

        t = srivastava(sample)
        # if not valid:
        if np.abs(t) > norm.ppf(1-(alpha/2)):
            rejected += 1

    emp_power = rejected/k

    result = {'c': cor, 'm': m, 'n': n, 'k': k, 'a': alpha, 'ep': emp_power}
    results = results.append(result, ignore_index=True)

results = results.loc[:,['ep', 'a', 'n', 'c', 'm', 'k']]

In [ ]:
results['n'] = results['n'].astype(int)
results['m'] = results['m'].astype(int)
results['k'] = results['k'].astype(int)

In [ ]:
results.to_csv('../text/latex/algo/files/test_validation_power.csv', index=False, float_format='%.3f')